Ao aplicar Análise Exploratória de Dados vou responder a estas 10 perguntas:

- 1- Quais São as Categorias de Filmes Mais Comuns no IMDB?
- 2- Qual o Número de Títulos Por Gênero?
- 3- Qual a Mediana de Avaliação dos Filmes Por Gênero?
- 4- Qual a Mediana de Avaliação dos Filmes Em Relação ao Ano de Estréia?
- 5- Qual o Número de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia?
- 6- Qual o Filme Com Maior Tempo de Duração? Calcule os Percentis.
- 7- Qual a Relação Entre Duração e Gênero?
- 8- Qual o Número de Filmes Produzidos Por País?
- 9- Quais São os Top 10 Melhores Filmes?
- 10- Quais São os Top 10 Piores Filmes?

# Instalando e Carregando Pacotes

In [1]:
# Versão de  linguagem Python
from platform import python_version
print('Versão da Python usada Neste Jupyter Notebook:', python_version())

Versão da Python usada Neste Jupyter Notebook: 3.8.8


In [2]:
#instalar pacote
!pip install -q imdb-sqlite

In [3]:
#instalar o pacote com os paises.
!pip install -q pycountry

In [4]:
import re #pacete de expreção regulares em texto - processa texto
import time #para medir o tempo de exec ou trab com datas
import sqlite3 #pacote para manipular banco de dados
import pycountry#pacote com os paises
import numpy as np#manipulação de dados
import pandas as pd#manipulação de dados
import matplotlib.pyplot as plt #visualização de dados graficos
import seaborn as sns# visualização de dados graficos
from matplotlib import cm# visualização de dados graficos
from sklearn.feature_extraction.text import CountVectorizer#"skylearn" machine learnig python.
import warnings# filtra qualquer tipo warnings
warnings.filterwarnings("ignore")#filtra qualquer tipo de warnings
sns.set_theme(style ="whitegrid")#definição de estilo 'tema' Graficos

# Carregando os Dados
#### primeiro temos que baixa os dados!

In [5]:
%%time
!imdb-sqlite

Wall time: 661 ms


2021-07-30 15:10:39,128 DB already exists: (imdb.db). Refusing to modify. Exiting


In [6]:
#conectar no banco de dados
conn = sqlite3.connect("imdb.db")

In [7]:
# Extrai a lista de tabelas
tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

In [8]:
# tipo do objeto
type(tabelas)

pandas.core.frame.DataFrame

In [9]:
# visualizar tabela
tabelas.head()

,Table_Name
0,people
1,titles
2,akas
3,crew
4,episodes


In [10]:
#convertendo essa tabela"dataframe" em lista
tabelas = tabelas['Table_Name'].values.tolist()

In [11]:
# Vamos percorrer a lista de tabelas no banco de dados e extrair o esquema de cada uma
for tabela in tabelas:
    consulta = "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conn)
    print("Esquema da tabela:", tabela)
    display(resultado)
    print("-"*100)
    print("\n")

Esquema da tabela: people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: akas


...

----------------------------------------------------------------------------------------------------


Esquema da tabela: episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




# 1- Quais são as Categorias de Filmes mais Comuns no IMDB?
##### Quais são as principais tipos (Categorias) dos títulos (filmes)?

In [12]:
#Cria a consulta SQL
consulta1 = '''Select type, COUNT(*) AS COUNT FROM titles GROUP BY type'''

In [13]:
# Extrai o resultado
resultado1 = pd.read_sql_query(consulta1,conn)

In [14]:
# Visualizar o resultado
display(resultado1)

,type,COUNT
0,movie,583821
1,radioEpisode,1
2,radioSeries,1
3,short,823381
4,tvEpisode,5952182
5,tvMiniSeries,38551
6,tvMovie,132355
7,tvSeries,211741
8,tvShort,10157
9,tvSpecial,33516


In [15]:
# Vamos calcular o percentual para cada tipo
resultado1['percentual'] = (resultado1['COUNT'] / resultado1['COUNT'].sum()) * 100

In [16]:
# Visualiza o resultado
display(resultado1)

,type,COUNT,percentual
0,movie,583821,7.180719
1,radioEpisode,1,0.000012
2,radioSeries,1,0.000012
3,short,823381,10.127192
4,tvEpisode,5952182,73.208987
5,tvMiniSeries,38551,0.474159
6,tvMovie,132355,1.627903
7,tvSeries,211741,2.604313
8,tvShort,10157,0.124926
9,tvSpecial,33516,0.412231


In [17]:
# Vamos cria um gráfico com apenas 4 categorias:
# As 3 categorias com mais títulos e 1 categoria com todo o restante

#Cria um dicionário vazio
others = {}

#Filtra o percentual em 5% e soma o total
others['COUNT'] = resultado1[resultado1['percentual'] < 5]['COUNT'].sum()

# Grava o percentual
others['percentual'] = resultado1[resultado1['percentual'] < 5]['percentual'].sum()

#Ajusta o nome
others['type'] = 'others'

In [18]:
# Visualiza
others

{'COUNT': 771014, 'percentual': 9.483102795213716, 'type': 'others'}

In [20]:
# Filtra o dataframe de resultado
resultado1 = resultado1[resultado1['percentual']> 5]

In [21]:
# Append com o dataframe de outras categorias
resultado1 = resultado1.append(others, ignore_index = True)

In [22]:
# Ordena o resultado
resultado1 = resultado1.sort_values(by = 'COUNT', ascending = False)